In [15]:
import pandas as pd
import chardet
import pandas as pd
import numpy as np
import jieba as jb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from gensim import corpora
import pyLDAvis.gensim_models
df = pd.read_excel('活頁簿1.xlsx')
df=df[['歌名','鑑賞詞性']]
print(len(df))
print(df.sample(10))

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1631
           歌名                                               鑑賞詞性
607   聽見下雨的聲音  這部 電影 音樂 關係 方文山 周傑倫 打造 同名 主題 - - - 聽見 下雨 聲音 抒情...
1238       巨鹿  和聲 安排 特別 新意 華晨 人聲 有意 採用 唱法 嗓音 模仿 動物 柔和 華美 鳴叫聲 ...
1444       句號  句號 整首歌 充滿 傷感 惋惜 歌詞 亮點 鄧紫棋 經歷 歌手 生涯 唱片 一馬當先 旁邊 ...
630    以後別做朋友  三金 音樂 神手 陳建騏 領軍 製作 網 封為 療傷 神麯 周興哲 傷感 嗓音 歌詞 悲傷感...
533        落俗  落俗 弦樂 演奏 紛繁 李榮浩 唱法 層次 對號入座 演繹 一節 張弛 聲音 之間 搶奪 聽...
550      停止跳動  傳統 情歌 汪蘇 電子 元素 多種 效果 音效 整首歌 氣場 與眾不同 歌詞 注重 細節 文...
1009     全面淪陷  難度 音域 好唱 高音 歌詞 男女之間 愛情 那種 淡淡 純純 整張 專輯 輕快的 最讓人 ...
237      黑暗騎士                   人生 關卡 傳達 信念 努力 曙光 象徵 音樂 騎士 拯救 世界
816     荊棘裡的花  音樂 內容 涵蓋 雷鬼 期望 透過 富有 人文 意涵 歌詞 探究 愛情 現象 宗教 心理 哲...
1088     面具舞會  張狂 個性 節奏感 首歌曲 融入 打碟 元素 旋律 心妍 再一次 迎接 曲風 挑戰 搖擺 雀...


In [16]:
df['label_id'] = df['歌名'].factorize()[0]
label_id_df = df[['歌名', 'label_id']].drop_duplicates().sort_values('label_id').reset_index(drop=True)
print(label_id_df)

           歌名  label_id
0         我難過         0
1         一直走         1
2        天若有情         2
3      以前, 以後         3
4        失戀無罪         4
...       ...       ...
1597       無眠      1597
1598  當我們一起走過      1598
1599       蟬想      1599
1600       那誰      1600
1601     知否知否      1601

[1602 rows x 2 columns]


C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
df['cut_text'] = df['鑑賞詞性']
print(df.head())

       歌名                                               鑑賞詞性  label_id  \
0     我難過  偶像劇 搭配 情人 熱播 難過 觀眾 熟悉 當年 罵聲 掌聲 時間 年級 成名 鄉民 稱作 ...         0   
1     一直走  擷取 獨特 節奏 音色 元素 入耳 感受到 黃麗玲 歡暢 情緒 爽朗 愉悅 電音 節拍 包裹...         1   
2    天若有情  情歌 美譽 歌聲 感情 拿捏 精準 到位 主歌 青衣 刻骨銘心 中音 傳達 細微 悲傷 雨水...         2   
3  以前, 以後  姚若龍 細碎 歌詞 筆觸 流露 純熟 女人味 坦然 心情 已成 未知 經 傾吐 黃麗玲 大氣...         3   
4    失戀無罪  首波 主打歌 失戀 無罪 最佳 作詞人 林夕 量身 訂做 送給 男友 劈腿 藉由 親身經驗 ...         4   

                                            cut_text  
0  偶像劇 搭配 情人 熱播 難過 觀眾 熟悉 當年 罵聲 掌聲 時間 年級 成名 鄉民 稱作 ...  
1  擷取 獨特 節奏 音色 元素 入耳 感受到 黃麗玲 歡暢 情緒 爽朗 愉悅 電音 節拍 包裹...  
2  情歌 美譽 歌聲 感情 拿捏 精準 到位 主歌 青衣 刻骨銘心 中音 傳達 細微 悲傷 雨水...  
3  姚若龍 細碎 歌詞 筆觸 流露 純熟 女人味 坦然 心情 已成 未知 經 傾吐 黃麗玲 大氣...  
4  首波 主打歌 失戀 無罪 最佳 作詞人 林夕 量身 訂做 送給 男友 劈腿 藉由 親身經驗 ...  


C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
no_features = 1000


tfidf = TfidfVectorizer(max_features=no_features)
tfidf_features = tfidf.fit_transform(df.cut_text)
tfidf_feature_names = tfidf.get_feature_names()


cv = CountVectorizer(max_features=no_features)
cv_features = cv.fit_transform(df.cut_text)
cv_feature_names = cv.get_feature_names()

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
#NMF
nmf_tfidf = NMF(n_components=15, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_features)

#LDA
lda_cv = LatentDirichletAllocation(n_components=15, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(cv_features)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("分類 {} : {}".format(topic_idx,"|".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])))

no_top_words = 10
print('---------------NMF-tfidf_features 主題-----------------------------------------')
display_topics(nmf_tfidf, tfidf_feature_names, no_top_words)
print()
print('--------------Lda-CountVectorizer_features 主題--------------------------------')
display_topics(lda_cv, cv_feature_names, no_top_words)

---------------NMF-tfidf_features 主題-----------------------------------------
分類 0 : 歌詞|情感|聲音|演繹|情緒|演唱|歌聲|充滿|內心|聽眾
分類 1 : 力量|希望|能量|鼓勵|傳遞|創作|勇敢|溫暖|勇氣|帶給
分類 2 : 生活|平凡|心中|平淡|幸福|發現|工作|日子|場景|態度
分類 3 : 描述|戀情|心情|歌詞|美好|柔軟|等待|感覺|風格|擁有
分類 4 : 專輯|概念|主打|整張|同名|主打歌|創作|演唱|歌迷|收錄
分類 5 : 故事|講述|孤獨|主角|回憶|電影|發生|背後|男生|導演
分類 6 : 簡單|創作|歌詞|乾淨|溫馨|直白|作品|鋼琴|熱愛|象徵
分類 7 : 節奏|搖滾|元素|風格|音樂|電子|歌詞|感覺|輕快|電音
分類 8 : 時間|時光|之間|歲月|變化|回憶|朋友|父母|身邊|距離
分類 9 : 世界|帶來|人生|守護|希望|改變|音樂|天使|視角|暗示
分類 10 : 愛情|戀人|階段|幸福|男女|兩人|女生|相愛|之間|甜蜜
分類 11 : 感情|分手|分開|逝去|回憶|這段|描寫|祝福|結束|男生
分類 12 : 夢想|放棄|努力|追求|成員|追尋|追夢|鼓勵|熱血|主題
分類 13 : 展現|呈現|魅力|兩人|聽眾|完美|方式|張碧晨|唱功|之間
分類 14 : 情歌|甜蜜|分手|周傑倫|勵志|傷感|典型|表現|悲傷|情侶

--------------Lda-CountVectorizer_features 主題--------------------------------
分類 0 : 時刻|公司|唱片|經驗|三人|大膽|歌迷|相遇|故事|釋懷
分類 1 : 戀人|愛情|女生|在一起|答案|尋找|歌詞|階段|深愛|終於
分類 2 : 歌詞|愛情|情感|感情|情歌|故事|情緒|聲音|演繹|簡單
分類 3 : 感覺|輕鬆|戀情|性感|那種|描述|意味|柔軟|美好|嗓音
分類 4 : 飽含|心路歷程|完美|魅力|理解|走來|完美的|三人|用心|女人
分類 5 : 歌詞|音樂|節奏|作品|風格|感覺|聽眾|情緒|充滿|世界
分類 6 : 嗓音|演繹|聲音|展現|男女|風格|浪漫|歌詞|魅力|演唱
分類 7 : 男生|故事|女生|那

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
import jieba.posseg as psg
import numpy
from collections import Counter
import openpyxl
import jieba

jieba.set_dictionary('dict.txt.big')
jieba.load_userdict('userdict.txt') 
with open('stops.txt', 'r', encoding='utf8') as f: 
    stops = f.read().split('\n') 
data=[]
data1=[]
allsong=[]
wb = openpyxl.load_workbook('活頁簿1.xlsx')
wb1 = openpyxl.load_workbook('詞性.xlsx')
ws = wb.active
ws1 = wb1.active
myList = [row for row in ws.values]
myList1 = [row for row in ws1.values]
myArr = numpy.array(myList)
myArr1 = numpy.array(myList1)
for i in range(1,1632):
    lyric=myArr[i][4]
    content=[]
    new_content=[]
    song=[]
    stops.append('\n\n') 
    for sent in lyric.split(): 
        filtered= [t for t in jieba.cut(sent) if t not in stops and t.split() and len(t)>1 ]# jieba.cut()回傳generator，要用forloop取值
        content+=filtered
        for i in content:
            if i not in new_content:
                new_content.append(i)
    #print(new_content)
    for j in range(len(new_content)):
        seg = psg.cut((new_content[j])) 
        for ele in seg:
            #print(ele)
            a='/'.join(ele)
                #print(type(ele))
            c=a.split("/")
            #print(c)結巴斷詞器的詞性標註分析
            for i in range(len(myArr1)):
                if(c[1]==myArr1[i][0]):
                    song.append(c[0])
    #print(song)
    data.append(song)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Building prefix dict from C:\Users\User\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.u7bdefbdcea8835a8c0038be7dab8bef4.cache
Loading model cost 1.817 seconds.
Prefix dict has been built successfully.


In [22]:
import re
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
def predict_topic_by_cv(text):
    #txt = remove_punctuation(text)
    #txt = [w for w in list(jb.cut(txt)) if w not in stopwords]
    #txt = [w for w in txt if len(w)>1]
    #txt = [" ".join([w for w in txt])]
    #a=["專輯","音樂","概念","私語","唱歌","對話","希望","著這","傳遞","樂觀","能量","鼓勵","所有人","勇敢"]
    newfeature = cv.transform(text)
    doc_topic_dist_unnormalized = np.matrix(lda_cv.transform(newfeature))
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    topicIdx = doc_topic_dist.argmax(axis=1)[0,0]
    #print('該文檔屬於:主題 {}'.format(topicIdx))
    #print("主題 {} : {}".format(topicIdx,"|".join([cv_feature_names[i] for i in (lda_cv.components_[topicIdx,:]).argsort()[:-no_top_words - 1:-1]])))
    return topicIdx
a=[]      
b=[]      
c=[]      
d=[]      
e=[]
f=[]
j=[]
i=[]
o=[]
t=[]
g=[]
h=[]
n=[]
s=[]
m=[]
r=[]
l=[]
q=[]
k=[]
p=[]
dict={0:a,1:b,2:c,3:d,4:e,5:f,6:g,7:h,8:i,9:j,10:k,11:l,12:m,13:n,14:o,15:p,16:q,17:r,18:s,19:t}
for i in range(len(data)):
    a=predict_topic_by_cv(data[i])
    if(a in dict):
        dict[a].append(i)
    #print(data[i])
for i in range(0,15):
    print("總數:",len(dict[i]))
    print(i,":",dict[i])

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


總數: 755
0 : [1, 6, 15, 17, 21, 26, 27, 29, 31, 35, 40, 46, 47, 48, 50, 54, 56, 57, 58, 59, 61, 63, 71, 74, 75, 76, 79, 80, 83, 84, 85, 86, 88, 93, 94, 95, 96, 99, 102, 103, 108, 110, 111, 113, 114, 117, 118, 120, 121, 122, 123, 126, 128, 129, 130, 132, 133, 137, 139, 141, 145, 152, 156, 158, 159, 161, 167, 170, 181, 184, 185, 186, 191, 194, 195, 196, 197, 198, 203, 207, 208, 212, 213, 215, 217, 221, 222, 223, 224, 229, 230, 231, 233, 235, 238, 239, 244, 245, 247, 249, 250, 251, 252, 256, 261, 265, 267, 268, 270, 275, 276, 278, 279, 281, 283, 284, 285, 286, 287, 289, 290, 291, 292, 294, 297, 299, 300, 302, 304, 305, 306, 307, 313, 314, 316, 320, 323, 324, 326, 327, 329, 332, 338, 339, 344, 348, 354, 355, 356, 359, 360, 361, 364, 365, 368, 370, 376, 379, 381, 382, 383, 387, 388, 389, 392, 394, 395, 396, 400, 401, 402, 404, 407, 410, 411, 412, 414, 415, 416, 417, 418, 419, 423, 425, 429, 430, 432, 435, 436, 437, 439, 440, 443, 444, 450, 451, 452, 456, 458, 462, 464, 467, 468, 469, 471, 47

In [9]:
print(dict[1])

[78, 82, 125, 210, 253, 277, 345, 362, 422, 428, 445, 457, 479, 480, 487, 493, 500, 507, 539, 581, 587, 609, 635, 641, 717, 754, 768, 783, 795, 819, 918, 964, 1009, 1010, 1035, 1075, 1105, 1107, 1124, 1171, 1178, 1203, 1220, 1223, 1262, 1328, 1416, 1417, 1449, 1451, 1463, 1511, 1540, 1552, 1580]


C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
import openpyxl
import numpy
wb = openpyxl.load_workbook('活頁簿1.xlsx')
ws = wb.active
myList = [row for row in ws.values]
myArr = numpy.array(myList)
all1=[]
for i in range(len(dict[14])):
    song=[]
    a=dict[14][i]
    for j in range(0,6):
        song.append(myArr[a+1][j])
    all1.append(song)
#print(all1)

C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import csv
with open('lyrics.csv','w',encoding='utf-8',newline='') as f: #開啟輸出的CSV檔案
    writer=csv.writer(f) #建立CSV檔寫入器
    writer.writerow(['歌手','歌名','YouTube id','YouTube 網址','鑑賞斷詞','歌詞斷詞'])
    for item in all1:
        writer.writerow((column for column in item))
print('\n讀取 csv 檔')


讀取 csv 檔


C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
def predict_topic_by_cv(text):
    #txt = remove_punctuation(text)
    #txt = [w for w in list(jb.cut(txt)) if w not in stopwords]
    #txt = [w for w in txt if len(w)>1]
    #txt = [" ".join([w for w in txt])]
    #a=["專輯","音樂","概念","私語","唱歌","對話","希望","著這","傳遞","樂觀","能量","鼓勵","所有人","勇敢"]
    newfeature = cv.transform(text)
    doc_topic_dist_unnormalized = np.matrix(lda_cv.transform(newfeature))
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    topicIdx = doc_topic_dist.argmax(axis=1)[0,0]
    #print('該文檔屬於:主題 {}'.format(topicIdx))
    #print("主題 {} : {}".format(topicIdx,"|".join([cv_feature_names[i] for i in (lda_cv.components_[topicIdx,:]).argsort()[:-no_top_words - 1:-1]])))
    return topicIdx
a=["朋友"]
h=predict_topic_by_cv(a)
print(h)

13


C:\Users\User\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
